In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install --upgrade tensorflow

In [3]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Sequential

tf.__version__

'2.0.0'

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


In [ ]:
model = Sequential()
model.add(Conv2D(32, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10))



In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.fit(x_train, y_train, epochs=10)


In [ ]:
tf.saved_model.save(model, "./mymodel/001234")

In [ ]:
!tree ./mymodel

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("./mymodel/001234")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [6]:
from tflite_runtime.interpreter import Interpreter


In [7]:
interpreter = Interpreter("./converted_model.tflite")

In [8]:
interpreter.allocate_tensors()

In [ ]:
interpreter.get_input_details()

In [5]:
import numpy as np
z = np.copy(x_train[0])

In [11]:
tensor_index = interpreter.get_input_details()[0]['index']
input_tensor = interpreter.tensor(tensor_index)()[0]
input_tensor[:, :] = z

In [12]:
interpreter.invoke()
#output_details = interpreter.get_output_details()[0]

RuntimeError: There is at least 1 reference to internal data
      in the interpreter in the form of a numpy array or slice. Be sure to
      only hold the function returned from tensor() if you are using raw
      data access.